<a href="https://colab.research.google.com/github/Plusholic/Engineer_BigDataAnalysis/blob/main/BigData_exercise_history.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata/

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 485 (delta 66), reused 49 (delta 44), pack-reused 363
Receiving objects: 100% (485/485), 8.89 MiB | 8.64 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [143]:
import pandas as pd
import numpy as np

# 1유형 1번
# 결측치제거
data = pd.read_csv("/content/yemoonsaBigdata/datasets/Part3/301_housing.csv")
data.dropna(inplace=True)
# data.info()
train = data.loc[0:int((len(data)*7/10)),:] # , 70% 까지를 훈련 데이터로 추출
print('1번 답 : ',int(train['housing_median_age'].quantile(1/4)))

# 1유형 2번
data = pd.read_csv("/content/yemoonsaBigdata/datasets/Part3/302_worlddata.csv")
select_data = data[data['year'] == 2000]
del select_data['year'] # year 제거해주는거 중요
select_data = select_data.T
len(select_data[select_data > select_data.mean()].dropna())
print('2번 답 : ',len(select_data[select_data > select_data.mean()].dropna()))

# # 1유형 3번
data = pd.read_csv("/content/yemoonsaBigdata/datasets/Part3/303_titanic.csv")
idx = np.argmax(data.isna().sum())
print('3번 답 :', data.columns[idx])

1번 답 :  19
2번 답 :  76
3번 답 : Age


In [138]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C85,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,C123,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,B42,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [110]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 1유형 1번
# 결측치제거
train = pd.read_csv("/content/yemoonsaBigdata/datasets/Part3/304_travel_insurance_train.csv")
X_test = pd.read_csv("/content/yemoonsaBigdata/datasets/Part3/304_travel_insurance_test.csv")

categoric_ = ['FrequentFlyer', 'EverTravelledAbroad', 'Employment Type', 'GraduateOrNot']
numeric_ = ['ID', 'Age', 'AnnualIncome','FamilyMembers','ChronicDiseases']#,'TravelInsurance']

# train.describe()
for col in numeric_:
  train = train[(train[col] > train[col].mean() - 2 * train[col].std()) & (train[col] < train[col].mean() + 2 * train[col].std())]

train.info()

le = LabelEncoder()
for col in categoric_:
  # print(train[col].value_counts())
  # print(' ')
  train[col] = le.fit_transform(train[col])
  X_test[col] = le.fit_transform(X_test[col])


X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

from sklearn.model_selection import train_test_split


X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, stratify = y_train)

scaler = MinMaxScaler()
X_tr[numeric_] = scaler.fit_transform(X_tr[numeric_])
X_val[numeric_] = scaler.fit_transform(X_val[numeric_])
X_test[numeric_] = scaler.fit_transform(X_test[numeric_])

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

xgb = XGBClassifier()
xgb.fit(X_tr, y_tr)

rf = RandomForestClassifier()
rf.fit(X_tr, y_tr)

pred_proba_1 = xgb.predict_proba(X_val)[:,1]
pred_proba_2 = rf.predict_proba(X_val)[:,1]

from sklearn.metrics import roc_auc_score

score_1 = roc_auc_score(y_val, pred_proba_1)
score_2 = roc_auc_score(y_val, pred_proba_2)
print(score_1)
print(score_2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1365 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   1365 non-null   int64 
 1   Age                  1365 non-null   int64 
 2   Employment Type      1365 non-null   object
 3   GraduateOrNot        1365 non-null   object
 4   AnnualIncome         1365 non-null   int64 
 5   FamilyMembers        1365 non-null   int64 
 6   ChronicDiseases      1365 non-null   int64 
 7   FrequentFlyer        1365 non-null   object
 8   EverTravelledAbroad  1365 non-null   object
 9   TravelInsurance      1365 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 117.3+ KB
0.8347966989486377
0.8306327015110977


In [ ]:
# 스케일링 안한거 0.8423710291291404